# Merge ndvi split data sets and reformat them for further analysis
In order to do this you must have downloaded the ndvi data sets for trees with and wiothout watering.

In [9]:
import pandas as pd


In [10]:
def load_and_pivot(paths, has_watering):
    # Load and merge raw CSVs
    df = pd.concat(
        [pd.read_csv(p) for p in paths],
        ignore_index=True
    )

    # Pivot to wide format (one row per tree)
    df_wide = (
        df.pivot_table(
            index="tree_id",
            columns="month",
            values="NDVI",
            aggfunc="mean"
        )
        .rename(columns=lambda m: f"NDVI_m{m}")
        .reset_index()
    )

    # Ensure all months 3–9 exist
    all_months = [f"NDVI_m{m}" for m in range(3, 10)]
    df_wide = df_wide.reindex(columns=["tree_id"] + all_months)
    df_wide.columns.name = None

    # Add watering flag
    df_wide["has_watering"] = has_watering

    return df_wide


In [11]:
# ---- Watering data ----
watering_paths = [
    "data/ndvi_CWP_2022_watering_p1.csv",
    "data/ndvi_CWP_2022_watering_p2.csv",
]

df_watering = load_and_pivot(watering_paths, has_watering=True)

In [12]:
# ---- No-watering data ----
no_watering_paths = [
    "data/ndvi_CWP_2022_no_watering_p1.csv",
    "data/ndvi_CWP_2022_no_watering_p2.csv",
]

df_no_watering = load_and_pivot(no_watering_paths, has_watering=False)

In [13]:
# ---- Combine both ----
df_final = pd.concat(
    [df_watering, df_no_watering],
    ignore_index=True
)

print(df_final.head())

             tree_id   NDVI_m3   NDVI_m4   NDVI_m5   NDVI_m6   NDVI_m7  \
0  00008100:000c09d7  0.224754  0.268153  0.306351  0.276893  0.251684   
1  00008100:000c09e9  0.157686  0.199009  0.183150  0.220465  0.204459   
2  00008100:000c09ea  0.180522  0.211626  0.199039  0.230015  0.227425   
3  00008100:000c0a20  0.309235  0.346179  0.363654  0.332270  0.322522   
4  00008100:000c0a21  0.352739  0.492772  0.479558  0.477902  0.478470   

    NDVI_m8   NDVI_m9  has_watering  
0  0.203854  0.358751          True  
1  0.187151  0.210316          True  
2  0.216174  0.237977          True  
3  0.311283  0.398499          True  
4  0.457277  0.547352          True  


In [14]:
df_final.to_csv("data/ndvi_CWP_2022.csv", index=False)